# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import sys

In [10]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

# 1-2

In [11]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [12]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


** To figure out the 'black' and 'white resume amounts and call backs we grouped by race and call. Both has 2435 resumes, but black got 157 calls meanwhile white got 235 calls.

In [6]:
dfg = data.groupby(['race','call']).count()
dfg.head()

id    ad  education  ofjobs  yearsexp  honors  volunteer  \
race call                                                               
b    0.0   2278  2278       2278    2278      2278    2278       2278   
     1.0    157   157        157     157       157     157        157   
w    0.0   2200  2200       2200    2200      2200    2200       2200   
     1.0    235   235        235     235       235     235        235   

           military  empholes  occupspecific    ...      compreq  orgreq  \
race call                                       ...                        
b    0.0       2278      2278           2278    ...         2278    2278   
     1.0        157       157            157    ...          157     157   
w    0.0       2200      2200           2200    ...         2200    2200   
     1.0        235       235            235    ...          235     235   

           manuf  transcom  bankreal  trade  busservice  othservice  missind  \
race call                                                                      
b    0.0    2278      2278      2278   2278        2278        2278     2278   
     1.0     157       157       157    157         157         157      157   
w    0.0    2200      2200      2200   2200        2200        2200     2200   
     1.0     235       235       235    235         235         235      235   

           ownership  
race call             
b    0.0        2278  
     1.0         157  
w    0.0        2200  
     1.0         235  

[4 rows x 63 columns]

In [14]:
print ('Total resumes: {}'.format(len(data)))
bl = data[data.race == 'b']
wh = data[data.race == 'w']
print ('Total black resumes: {}'.format(len(bl.call)))
print ('Total white resumes: {}'.format(len(wh.call)))

Total resumes: 4870
Total black resumes: 2435
Total white resumes: 2435


**Since the size of the data is big enough and 'black' and 'white' values in race are assigned randomly to the resumes when presented to the employer, CLT could be applied here assuming that samples are respresenting the population.**

**We are supposed to apply the two sample z-test**
Null Hypotheses : H0 : proportion (black) = proportion (white) 
Alternate Hypotheses : HA : proportion (black) != proportion (white)

# 3 

In [15]:
bl_call_mean = bl.call.mean()
wh_call_mean = wh.call.mean()
print ('Black mean: {}'.format(bl_call_mean))
print ('White mean: {}'.format(wh_call_mean))
diff_mean = wh_call_mean - bl_call_mean
print ('Difference between White and Black mean: {}'.format(diff_mean))

Black mean: 0.0644763857126
White mean: 0.0965092405677
Difference between White and Black mean: 0.0320328548551


In [24]:
wh_cll = np.sum(data[data.race=='w'].call)
bl_cll = np.sum(data[data.race=='b'].call)
wh_ln = len(data[data.race=='w'])
bl_ln = len(data[data.race=='b'])            
phat = (wh_cll + bl_cll) / (wh_ln + bl_ln)
z = (diff_mean) / np.sqrt(phat * (1 - phat) * (( 1/ float(wh_ln)) + (1/float(bl_ln))))
print ('z score:{}'.format(z))
p = stats.norm.cdf(-z)*2
print ('p-value:{}'.format(p))

z score:4.10841223524
p-value:3.9838854095e-05


In [25]:
moe = 1.96 * np.sqrt(phat * (1 - phat) * (( 1/ float(wh_ln)) + (1/float(bl_ln))))
ci = diff_mean + np.array([-1, 1]) * moe
print("Margin of Error: {}".format(moe))
print("Confidence interval: {}".format(ci))

Margin of Error: 0.0152819123109
Confidence interval: [ 0.01675094  0.04731477]


# 4

Since p-value is too close to 0 we can assumed to be 0. Therefore the null hypothesis should be rejected. They don't have the same call back rate.
The analysis we made does not mean that race is the most crucial factor in callback success. Despite the test shows that callback rates differs, other variables as such year of experience, education or maybe military statistics might effect the callback rates and need to be also checked. However, this example shows that a significant difference exist.

In [26]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2, pval, _, _ = stats.chi2_contingency(cont_table)
print("Chi-squared test statistic: {}".format(chi2))
print("p-value: {}".format(pval))

Chi-squared test statistic: 16.4490285842
p-value: 4.99757838996e-05


**X2 test yields the same result.**